In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import datetime
import os

In [12]:
# 네이버 시리즈 카테고리별 링크 추출

base_url = 'https://series.naver.com'
url = 'https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=205&orderTypeCode=sale&is&isFinished=false&page={}'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

data = []
for i in range(1, 39):
  response = requests.get(url.format(i), headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')

  lst_list = soup.select('ul.lst_list > li')
  for lst in lst_list:
    link = lst.select_one('div > h3 > a')['href']
    data.append(base_url+link)

df = pd.DataFrame({'link':data})
df.to_csv('genre_205.csv', index=False)

In [13]:
df1 = pd.read_csv('genre.csv')
df2 = pd.read_csv('genre_202.csv')
df3 = pd.read_csv('genre_203.csv')
df4 = pd.read_csv('genre_205.csv')
df5 = pd.read_csv('genre_206.csv')
df6 = pd.read_csv('genre_207.csv')
df7 = pd.read_csv('genre_208.csv')

df = pd.concat([df1, df2, df3, df4, df5, df6, df7])
df.to_csv('links.csv')

In [5]:
options = Options()
options.add_argument("--headless") 
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")

driver = webdriver.Chrome(options=options)

datas = []

df = pd.read_csv('links.csv')

for link in df['link']:
  driver.get(link)
  time.sleep(1)

  html = driver.page_source

  soup = BeautifulSoup(html, 'html.parser')
  content = soup.find('#content')

  data = {}
  try:
    data['title'] = soup.select_one('div.end_head > h2').get_text()
    data['u_cnt'] = soup.find('em', class_='u_cnt').get_text()
    data['reviewCount'] = soup.find('span', id='reviewCount').get_text()

    lists = soup.select('li.info_lst > ul > li')
    data['ing'] = lists[0].get_text()
    data['genre'] = lists[1].get_text()
    data['author'] = lists[2].find('a').get_text()
    data['publisher'] = soup.find('span', text='출판사').find_next_sibling().get_text()
    data['users'] = lists[-1].get_text()

    free_info = soup.select_one('div.area_text_information > strong')
    data['free_info'] = free_info.get_text() if free_info else '0화'

    data['uploaded'] = soup.select_one('h5.end_total_episode > strong').get_text()

    recomms = soup.select('em.u_cbox_cnt_recomm')
    unrecomms = soup.select('em.u_cbox_cnt_recomm')

    data['reaction'] = 0

    for i in range(len(recomms)):
      recomm = int(recomms[i].get_text().strip())
      unrecomm = int(unrecomms[i].get_text().strip())

      data['reaction'] += (recomm + unrecomm)

    datas.append(data)

  except Exception as e:
    print("Error by", e)
    print(link)
    continue

driver.quit()

df1 = pd.DataFrame(datas)
df1.to_csv('test.csv', index=False)
df1

C:\Users\tkdeh\AppData\Local\Temp\ipykernel_13688\3037969273.py:30: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  data['publisher'] = soup.find('span', text='출판사').find_next_sibling().get_text()


Error by 'NoneType' object has no attribute 'get_text'
https://series.naver.com/novel/detail.series?productNo=1085514
Error by 'NoneType' object has no attribute 'get_text'
https://series.naver.com/novel/detail.series?productNo=1937734
Error by 'NoneType' object has no attribute 'get_text'
https://series.naver.com/novel/detail.series?productNo=1594024
Error by 'NoneType' object has no attribute 'get_text'
https://series.naver.com/novel/detail.series?productNo=1621228
Error by 'NoneType' object has no attribute 'get_text'
https://series.naver.com/novel/detail.series?productNo=1279681
Error by 'NoneType' object has no attribute 'get_text'
https://series.naver.com/novel/detail.series?productNo=1930223
Error by 'NoneType' object has no attribute 'get_text'
https://series.naver.com/novel/detail.series?productNo=1553176
Error by 'NoneType' object has no attribute 'get_text'
https://series.naver.com/novel/detail.series?productNo=1669068
Error by 'NoneType' object has no attribute 'get_text'
h